In [1]:
!pip -q install pymongo datasets transformers accelerate peft bitsandbytes trl evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 22.9 MB/s eta 0:00:00


In [ ]:
import os
from pymongo import MongoClient
from datasets import Dataset, DatasetDict

client = MongoClient("")

db = client["ai_evaluation"]

train_col = db["train_data"]    # change if your name differs
test_col  = db["test_data"]           # change if your name differs

def load_collection(col, limit=None):
    cursor = col.find({}, {"_id": 0})
    if limit:
        cursor = cursor.limit(limit)
    return list(cursor)

train_docs = load_collection(train_col)
test_docs  = load_collection(test_col)

train_ds = Dataset.from_list(train_docs)
test_ds = Dataset.from_list(test_docs)


In [ ]:
train_ds[20]

{'question': 'Bella bought stamps at the post office. Some of the stamps had a snowflake design, some had a truck design, and some had a rose design. Bella bought 11 snowflake stamps. She bought 9 more truck stamps than snowflake stamps, and 13 fewer rose stamps than truck stamps. How many stamps did Bella buy in all?',
 'answer': 'How many truck stamps did Bella buy? ** The number of truck stamps is 11 + 9 = <<11+9=20>>20.\nHow many rose stamps did Bella buy? ** The number of rose stamps is 20 − 13 = <<20-13=7>>7.\nHow many stamps did Bella buy in all? ** Bella bought 11 + 20 + 7 = <<11+20+7=38>>38 stamps in all.\n#### 38'}

In [3]:
import re

ANS_RE = re.compile(r"(.*?)(####\s*(-?[0-9\.,]+))", re.DOTALL)

def split_answer(example):
    text = example["answer"]

    m = ANS_RE.search(text)

    if m:
        example["explanation"] = m.group(1).strip()
        example["answer"] = "#### " + m.group(3).replace(",", "").strip()
    else:
        # explicitly mark broken rows
        example["explanation"] = text.strip()
        example["answer"] = None

    return example


train_ds = train_ds.map(split_answer)
test_ds  = test_ds.map(split_answer)



Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [4]:
train_ds[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': '#### 72',
 'explanation': 'How many clips did Natalia sell in May? ** Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nHow many clips did Natalia sell altogether in April and May? ** Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.'}

In [5]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `Test` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushi

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm

In [17]:
def format_prompt(question):
    return f"""
You are a careful math reasoning assistant.

Rules:
1. Solve the problem step by step.
2. You MUST finish with a final answer.
3. The final line MUST be exactly:
#### <number>

Do NOT stop early.

Problem:
{question}
"""


In [20]:
def generate_answer(question, max_new_tokens=1500):
    prompt = format_prompt(question)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            repetition_penalty=1.05,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(out[0], skip_special_tokens=True)


In [21]:
import re

def extract_answer_strict(text):
    m = re.search(r"####\s*(-?[0-9\.,]+)", text)
    return m.group(1).replace(",", "") if m else None

def extract_answer_relaxed(text):
    if not text:
        return None

    strict = extract_answer_strict(text)
    if strict is not None:
        return strict

    m = re.search(r"\\boxed\{(-?[0-9]+(?:\.[0-9]+)?)\}", text)
    if m:
        return m.group(1)

    return None

def normalize_number(s):
    if s is None:
        return None
    s = s.strip().replace(",", "")
    if s.endswith(".0"):
        s = s[:-2]
    if s.endswith("."):
        s = s[:-1]
    return s

def classify_error(model_output, gt):
    if extract_answer_strict(model_output) is None:
        if "\\boxed" in model_output:
            return "formatting_error"
        return "missing_final_answer"

    pred = normalize_number(extract_answer_strict(model_output))
    if pred != gt:
        return "reasoning_error"

    return "correct"





In [22]:
from tqdm import tqdm

zero_shot_col = db["zero_shot_results"]

def zero_shot_eval_and_store(ds, limit=None):
    strict_correct_count = 0
    numeric_correct_count = 0
    total = 0
    to_insert = []

    iterable = ds if limit is None else ds.select(range(min(limit, len(ds))))

    for ex in tqdm(iterable):
        q = ex["question"]
        gt_text = ex["answer"]

        out = generate_answer(q)

        # --- Ground truth ---
        gt = normalize_number(extract_answer_strict(gt_text))

        # --- Predictions ---
        strict_pred  = normalize_number(extract_answer_strict(out))
        relaxed_pred = normalize_number(extract_answer_relaxed(out))

        # --- Correctness ---
        strict_correct = (
            strict_pred is not None and
            gt is not None and
            strict_pred == gt
        )

        numeric_correct = (
            relaxed_pred is not None and
            gt is not None and
            relaxed_pred == gt
        )

        error_type = "correct" if strict_correct else classify_error(out, gt)

        to_insert.append({
            "question": q,
            "ground_truth_answer": gt,

            # strict (benchmark)
            "model_answer": strict_pred,
            "strict_correct": strict_correct,

            # relaxed (analysis)
            "model_answer_relaxed": relaxed_pred,
            "numeric_correct": numeric_correct,

            "model_output": out,
            "error_type": error_type,
            "mode": "zero_shot",
            "model_name": "Qwen2.5-3B-Instruct",
            "quantization": "4bit-nf4"
        })

        strict_correct_count += int(strict_correct)
        numeric_correct_count += int(numeric_correct)
        total += 1

    if to_insert:
        zero_shot_col.insert_many(to_insert)

    print(
        f"Strict accuracy:  {strict_correct_count}/{total} = {strict_correct_count/total:.3f}\n"
        f"Numeric accuracy: {numeric_correct_count}/{total} = {numeric_correct_count/total:.3f}"
    )


In [24]:
zero_shot_eval_and_store(test_ds, limit =300 )


100%|██████████| 300/300 [1:32:42<00:00, 18.54s/it]


Strict accuracy:  190/300 = 0.633
Numeric accuracy: 231/300 = 0.770
